In [ ]:
# !python -m pip install pandas
# !python -m pip install pyodbc


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
import pyodbc

In [33]:

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au\ODS;'
    'DATABASE=paragon;'
    'Trusted_Connection=yes;'  # 如果你用的是 Windows 身份验证
)

<>:3: SyntaxWarning: "\O" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\O"? A raw string is also an option.
<>:3: SyntaxWarning: "\O" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\O"? A raw string is also an option.
C:\Users\zengsh\AppData\Local\Temp\ipykernel_7860\3271171590.py:3: SyntaxWarning: "\O" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\O"? A raw string is also an option.
  'SERVER=prdsql05.westfund.com.au\ODS;'


In [ ]:
# query = "SELECT * FROM [paragon].[dbo].[group_key_full_by_branch]"
# df = pd.read_sql(query, conn)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_8476\122807158.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# 设置目标文件夹路径（请根据你的实际路径修改）
folder_path = "C:/Users/zengsh/OneDrive - Westfund Ltd/Documents/Westfund_/Vscode_/sql_db/ODC/movement"

# 如果文件夹不存在则创建
os.makedirs(folder_path, exist_ok=True)

# 获取今天的日期并格式化为 dd_mm_yyyy
file_date = datetime.today().strftime('%d_%m_%Y')

# 构建完整文件路径
file_name = f"movement_{file_date}.csv"
full_path = os.path.join(folder_path, file_name)

# 保存为 CSV 文件（覆盖同名文件）
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"DataFrame 已保存为 CSV 文件：{full_path}")

In [2]:
# 第一个变量：文件夹路径
folder_path = "C:/Users/zengsh/OneDrive - Westfund Ltd/Documents/Westfund_/Vscode_/sql_db/claims"
# 第二个变量：文件名（带日期）
file_date = datetime.today().strftime('%d_%m_%Y')
# file_date = '03_11_2025'
file_name = f"claims_lookup_silver_{file_date}.csv"
# file_name_2 = f"claims_OG_{file_date}.csv"
file_name_3 = f"claims_detailed_silver_{file_date}.csv"

# 合并成完整路径
full_path = os.path.join(folder_path, file_name)
# full_path_og = os.path.join(folder_path, file_name_2)
full_path_detailed = os.path.join(folder_path, file_name_3)

# 读取 CSV 文件
df = pd.read_csv(full_path, encoding='utf-8-sig', low_memory=False)
# df_og = pd.read_csv(full_path_og, encoding='utf-8-sig', low_memory=False)
df_dt = pd.read_csv(full_path_detailed, encoding='utf-8-sig', low_memory=False)


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17054804 entries, 0 to 17054803
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   claim_id       int64 
 1   membership_id  int64 
 2   cover_state    object
 3   cover_type     object
 4   cover_version  int64 
 5   product_id     int64 
dtypes: int64(4), object(2)
memory usage: 780.7+ MB


In [13]:
df.head()

,claim_id,membership_id,cover_state,cover_type,cover_version,product_id
0,32260892,144261,QLD,C,5,141
1,32352905,21716,NSW,F,2,11
2,32369961,52649,QLD,F,3,6
3,32121694,116270,NSW,F,5,29
4,32190340,23699,NSW,C,11,30


In [ ]:
filtered_df = df[df['claim_id'].isin([31189761, 31510321])].sort_values(by='claim_id')

In [17]:
filtered_df.sort_values(by='claim_id', ascending=True)

,claim_id,membership_id,cover_state,cover_type,cover_version,product_id
7034281,31189761,223726,QLD,F,1,157
7208962,31189761,223726,QLD,F,1,179
1219403,31510321,228781,NSW,S,1,185
11730091,31510321,228781,NSW,S,1,156


In [3]:
df_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129221 entries, 0 to 129220
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   service_date   129221 non-null  object 
 1   paid_date      129221 non-null  object 
 2   claim_type     129221 non-null  object 
 3   product_id     129221 non-null  int64  
 4   cover_state    128574 non-null  object 
 5   cover_type     128574 non-null  object 
 6   serv           129221 non-null  int64  
 7   fee_amt        129221 non-null  float64
 8   benefit_amt    129221 non-null  float64
 9   gross_deficit  129221 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 9.9+ MB


In [4]:
date_columns = ['service_date','paid_date']

for col in date_columns:
    print(f"原始类型 - {col}: {df_og[col].dtype}")
    # 自动解析日期，错误的值会变成 NaT
    df_og[col] = pd.to_datetime(df_og[col], errors='coerce')
    print(f"转换后类型 - {col}: {df_og[col].dtype}")

# 2️⃣ 检查是否有无法解析的日期
print("\n缺失日期统计")
print(df_og[date_columns].isna().sum())

# 3️⃣ 检查日期范围
print("\n日期范围检查")
for col in date_columns:
    print(f"{col} 最小值: {df_og[col].min()}, 最大值: {df_og[col].max()}")

原始类型 - service_date: object
转换后类型 - service_date: datetime64[ns]
原始类型 - paid_date: object
转换后类型 - paid_date: datetime64[ns]

缺失日期统计
service_date    0
paid_date       0
dtype: int64

日期范围检查
service_date 最小值: 2007-06-30 00:00:00, 最大值: 2025-10-31 00:00:00
paid_date 最小值: 2022-07-31 00:00:00, 最大值: 2025-10-31 00:00:00


In [5]:
df_og.head()

,service_date,paid_date,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
0,2024-11-30,2024-12-31,M,83,NSW,C,62,9917.41,4254.65,2860.67655
1,2022-08-31,2022-09-30,M,184,WA,S,28,2874.80,1152.45,0.00000
2,2024-05-31,2024-06-30,H,148,NSW,F,28,37557.17,36807.17,2442.12800
3,2022-09-30,2022-10-31,M,152,VIC,F,6,1695.85,661.90,0.00000
4,2023-08-31,2024-01-31,M,148,QLD,F,5,219.80,54.90,0.00000


In [6]:
# 定义时间范围
start_date = '2025-09-01'
end_date = '2025-09-30'

# 过滤数据
mask = (df_og['service_date'] >= start_date) & (df_og['service_date'] <= end_date)
filtered_df = df_og[mask]

In [7]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 979 entries, 8 to 129220
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   service_date   979 non-null    datetime64[ns]
 1   paid_date      979 non-null    datetime64[ns]
 2   claim_type     979 non-null    object        
 3   product_id     979 non-null    int64         
 4   cover_state    974 non-null    object        
 5   cover_type     974 non-null    object        
 6   serv           979 non-null    int64         
 7   fee_amt        979 non-null    float64       
 8   benefit_amt    979 non-null    float64       
 9   gross_deficit  979 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(3)
memory usage: 84.1+ KB


In [8]:
filtered_df

,service_date,paid_date,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
8,2025-09-30,2025-10-31,M,184,QLD,S,2,145.35,75.09,31.91325
632,2025-09-30,2025-09-30,M,81,QLD,S,50,10647.54,3760.84,2825.30320
797,2025-09-30,2025-10-31,M,30,ACT,C,3,1440.00,93.55,65.48500
864,2025-09-30,2025-09-30,H,179,VIC,F,1,1163.00,413.00,0.00000
987,2025-09-30,2025-10-31,M,83,QLD,C,2,314.00,112.90,79.03000
...,...,...,...,...,...,...,...,...,...,...
129030,2025-09-30,2025-09-30,B,157,NSW,C,1,560.14,560.14,0.00000
129031,2025-09-30,2025-10-31,B,129,NSW,C,1,530.88,530.88,0.00000
129116,2025-09-30,2025-09-30,B,129,WA,C,1,1253.00,1253.00,0.00000
129219,2025-09-30,2025-09-30,B,129,NSW,F,1,480.72,480.72,0.00000


In [3]:
df_dt[['cover_state', 'cover_type']] = df_dt[['cover_state', 'cover_type']].fillna('NA')

In [4]:
df_dt['service_date'] = pd.to_datetime(df_dt['service_date'], errors='raise')
df_dt['paid_date'] = pd.to_datetime(df_dt['paid_date'], errors='raise')

In [5]:
df_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2383884 entries, 0 to 2383883
Data columns (total 12 columns):
 #   Column         Dtype         
---  ------         -----         
 0   service_date   datetime64[ns]
 1   paid_date      datetime64[ns]
 2   claim_id       int64         
 3   membership_id  int64         
 4   claim_type     object        
 5   product_id     int64         
 6   cover_state    object        
 7   cover_type     object        
 8   serv           float64       
 9   fee_amt        float64       
 10  benefit_amt    float64       
 11  gross_deficit  float64       
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 218.3+ MB


In [6]:
# 按照第一段 SQL 聚合的维度
agg_columns = ['service_date', 'paid_date', 'claim_type', 'product_id', 'cover_state', 'cover_type']

# 聚合数值列
df_agg = df_dt.groupby(agg_columns, as_index=False).agg({
    'serv': 'sum',
    'fee_amt': 'sum',
    'benefit_amt': 'sum',
    'gross_deficit': 'sum'
})

# 这样得到的 df_agg 就是类似第一段 SQL 的 result
df_agg.head()


,service_date,paid_date,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
0,2007-06-30,2023-02-28,A,0,NA,NA,-2.0,-100.0,-70.0,0.0
1,2007-07-31,2023-02-28,A,0,NA,NA,-1.0,-50.0,-35.0,0.0
2,2008-05-31,2023-02-28,A,0,NA,NA,-5.0,-250.0,-175.0,0.0
3,2008-07-31,2023-02-28,A,0,NA,NA,-1.0,-50.0,-35.0,0.0
4,2010-09-30,2024-02-29,H,0,NA,NA,-12.0,-31077.0,-31077.0,0.0


In [7]:
# 定义时间范围
start_date = '2025-09-01'
end_date = '2025-09-30'

# 过滤数据
mask = (df_agg['service_date'] >= start_date) & (df_agg['service_date'] <= end_date)
filtered_df = df_agg[mask]

In [8]:
filtered_df

,service_date,paid_date,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
128058,2025-09-30,2025-09-30,A,0,NA,NA,128.0,24695.13,8946.90,0.00
128059,2025-09-30,2025-09-30,A,0,NSW,F,6.0,749.00,250.00,0.00
128060,2025-09-30,2025-09-30,A,0,QLD,C,2.0,174.00,50.00,0.00
128061,2025-09-30,2025-09-30,A,0,QLD,S,1.0,123.50,25.00,0.00
128062,2025-09-30,2025-09-30,A,9,NSW,C,9.0,751.00,528.90,0.00
...,...,...,...,...,...,...,...,...,...,...
129032,2025-09-30,2025-10-31,M,184,NSW,C,3.0,67.32,36.47,25.53
129033,2025-09-30,2025-10-31,M,184,NSW,S,1.0,339.00,107.75,64.65
129034,2025-09-30,2025-10-31,M,184,QLD,S,2.0,145.35,75.09,31.91
129035,2025-09-30,2025-10-31,M,187,NSW,S,2.0,77.70,40.12,0.00


In [18]:
from Lib_Westfund import compare_columns,Logger

#Claim details Null handle, 0 product check

In [89]:
df

,claim_id,membership_id,cover_state,cover_type,cover_version,product_id
0,32260892,144261,QLD,C,5,141
1,32352905,21716,NSW,F,2,11
2,32369961,52649,QLD,F,3,6
3,32121694,116270,NSW,F,5,29
4,32190340,23699,NSW,C,11,30
...,...,...,...,...,...,...
17054799,32377757,236485,SA,C,1,158
17054800,32379138,57734,QLD,C,6,139
17054801,32381760,108745,QLD,F,2,6
17054802,32414990,140216,NSW,C,3,145


In [90]:
df_dt

,service_date,paid_date,claim_id,membership_id,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
0,2025-01-31,2025-01-31,31936845,192236,A,156,QLD,F,2.0,140.0,60.0,0.0
1,2024-02-29,2024-02-29,31168586,203476,A,156,QLD,P,5.0,302.0,302.0,0.0
2,2024-01-31,2024-01-31,31063789,81969,A,154,QLD,F,1.0,65.0,55.0,0.0
3,2025-02-28,2025-02-28,32061824,24607,A,154,NSW,C,3.0,0.0,600.0,0.0
4,2024-10-31,2024-10-31,31712484,217518,A,156,VIC,C,1.0,68.0,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2383879,2023-08-31,2023-09-30,30831847,89426,A,29,NSW,S,1.0,58.0,58.0,0.0
2383880,2023-08-31,2023-08-31,30737264,80023,A,155,QLD,S,1.0,15.0,10.0,0.0
2383881,2024-03-31,2024-03-31,31233508,164735,A,157,QLD,F,3.0,198.0,135.0,0.0
2383882,2024-07-31,2024-10-31,31724371,193236,A,155,NSW,F,4.0,0.0,77.0,0.0


In [92]:
# cover_state 统计，包括 null
cover_state_counts = df_dt.groupby('cover_state', dropna=False).size().reset_index(name='count')

# cover_type 统计，包括 null
cover_type_counts = df_dt.groupby('cover_type', dropna=False).size().reset_index(name='count')

# 查看结果
cover_state_counts
# cover_type_counts


,cover_state,count
0,ACT,22432
1,NSW,1152276
2,NT,1191
3,QLD,1008364
4,SA,44482
5,TAS,7629
6,VIC,103682
7,WA,39317
8,NaN,4511


In [93]:
def fill_nulls(df, fill_dict):
    """
    df: pandas DataFrame
    fill_dict: dict, key = column name, value = fill value

    功能：
    1. 输出各列 null 数量
    2. 填充缺失值
    3. 输出填充后的 null 数量
    """
    print("原始缺失值统计：")
    print(df[list(fill_dict.keys())].isnull().sum(), "\n")
    
    for col, value in fill_dict.items():
        df[col].fillna(value, inplace=True)
    
    print("填充后缺失值统计：")
    print(df[list(fill_dict.keys())].isnull().sum())
    
    return df

# 使用示例
fill_values = {
    'cover_state': 'NSW',
    'cover_type': 'S'
}

df_dt_clean_ = fill_nulls(df_dt, fill_values)



原始缺失值统计：
cover_state    4511
cover_type     4511
dtype: int64 

填充后缺失值统计：
cover_state    0
cover_type     0
dtype: int64


In [94]:
df_dt_clean_

,service_date,paid_date,claim_id,membership_id,claim_type,product_id,cover_state,cover_type,serv,fee_amt,benefit_amt,gross_deficit
0,2025-01-31,2025-01-31,31936845,192236,A,156,QLD,F,2.0,140.0,60.0,0.0
1,2024-02-29,2024-02-29,31168586,203476,A,156,QLD,P,5.0,302.0,302.0,0.0
2,2024-01-31,2024-01-31,31063789,81969,A,154,QLD,F,1.0,65.0,55.0,0.0
3,2025-02-28,2025-02-28,32061824,24607,A,154,NSW,C,3.0,0.0,600.0,0.0
4,2024-10-31,2024-10-31,31712484,217518,A,156,VIC,C,1.0,68.0,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2383879,2023-08-31,2023-09-30,30831847,89426,A,29,NSW,S,1.0,58.0,58.0,0.0
2383880,2023-08-31,2023-08-31,30737264,80023,A,155,QLD,S,1.0,15.0,10.0,0.0
2383881,2024-03-31,2024-03-31,31233508,164735,A,157,QLD,F,3.0,198.0,135.0,0.0
2383882,2024-07-31,2024-10-31,31724371,193236,A,155,NSW,F,4.0,0.0,77.0,0.0


In [95]:
# 定义时间范围
start_date = '2025-09-01'
end_date = '2025-09-30'

# 过滤数据
mask = (df_dt_clean_['service_date'] >= start_date) & (df_dt_clean_['service_date'] <= end_date)
df_dt_clean_filtered = df_dt_clean_[mask]

In [96]:
# 1️⃣ 过滤出 product_id = 0 的行
df_zero = df_dt_clean_filtered[df_dt_clean_filtered['product_id'] == 0]

# 2️⃣ 用 claim_id 和 membership_id 去另一个 DataFrame 匹配
df_matched = df_zero.merge(
    df[['claim_id', 'membership_id', 'product_id', 'cover_state', 'cover_type']],
    on=['claim_id', 'membership_id'],
    how='left',  # 如果你只想抓到 df_zero 中存在的
    suffixes=('_zero', '_lookup')
)

# 结果中包含：
# df_matched['product_id_other'], df_matched['cover_state'], df_matched['cover_type']


In [98]:
df_matched = df_matched.drop_duplicates()

In [99]:
df_matched

,service_date,paid_date,claim_id,membership_id,claim_type,product_id_zero,cover_state_zero,cover_type_zero,serv,fee_amt,benefit_amt,gross_deficit,product_id_lookup,cover_state_lookup,cover_type_lookup
0,2025-09-30,2025-09-30,32534566,237240,A,0,NSW,S,1.0,110.0,25.0,0.0,149,QLD,C
1,2025-09-30,2025-09-30,32534566,237240,A,0,NSW,S,1.0,110.0,25.0,0.0,157,QLD,C
2,2025-09-30,2025-09-30,32563280,227313,A,0,QLD,C,1.0,87.0,25.0,0.0,160,QLD,C
3,2025-09-30,2025-09-30,32563280,227313,A,0,QLD,C,1.0,87.0,25.0,0.0,177,QLD,C
5,2025-09-30,2025-09-30,32546237,232904,A,0,NSW,S,3.0,316.5,135.0,0.0,179,NSW,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,2025-09-30,2025-09-30,32568805,63516,A,0,NSW,S,1.0,8750.5,1750.0,0.0,141,NSW,S
543,2025-09-30,2025-09-30,32568805,63516,A,0,NSW,S,1.0,8750.5,1750.0,0.0,21,NSW,F
544,2025-09-30,2025-09-30,32568805,63516,A,0,NSW,S,1.0,8750.5,1750.0,0.0,30,NSW,C
545,2025-09-30,2025-09-30,32568805,63516,A,0,NSW,S,1.0,8750.5,1750.0,0.0,30,NSW,F
